In [1]:
## Standard Stuff
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import dask.array as da
import numcodecs

## HEALPix Specific
import healpix as hp
import easygems.healpix as egh
import easygems.remap as egr

import intake     # For catalogs
import zarr       # Data Formatting

In [2]:
catfn='/home/tmerlis/hackathon/xsh24_main.yaml'

xsh24_cat = intake.open_catalog(catfn)

# right now, have coarse and native (don't have time-independent bcs)
print (list(xsh24_cat)) 

['xsh24_coarse', 'xsh24_native']


In [14]:
zoom_select = 7
ds = xsh24_cat.xsh24_coarse(zoom=zoom_select).to_dask()
ds = ds.pipe(egh.attach_coords)
ds

# as of may5, no coarser than level 10 zooms for the native zarr store
dsn = xsh24_cat.xsh24_native(zoom=10).to_dask()
dsn = dsn.pipe(egh.attach_coords)
#dsn

/home/tmerlis/.conda/envs/easy25/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/home/tmerlis/.conda/envs/easy25/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [11]:
ds

<xarray.Dataset> Size: 61GB
Dimensions:        (time: 3560, cell: 12288, plev: 31, pfull: 79)
Coordinates:
  * pfull          (pfull) float32 316B 4.514 8.301 12.45 ... 989.5 994.3 998.3
  * plev           (plev) float32 124B 1.0 2.0 3.0 5.0 ... 950.0 975.0 1e+03
  * time           (time) object 28kB 2019-10-20 03:00:00 ... 2021-01-07 00:0...
    crs            int64 8B 0
  * cell           (cell) int64 98kB 8 24 40 56 ... 196552 196568 196584 196600
    lat            (cell) float64 98kB 3.583 8.386 8.386 13.25 ... nan nan nan
    lon            (cell) float64 98kB 42.19 47.81 36.56 ... 0.004055 0.001355
Data variables: (12/32)
    clivi          (time, cell) float32 175MB ...
    clt            (time, cell) float32 175MB ...
    clwvi          (time, cell) float32 175MB ...
    hflsd          (time, cell) float32 175MB ...
    hfssd          (time, cell) float32 175MB ...
    hur            (time, plev, cell) float32 5GB ...
    ...             ...
    uas            (time, cell) float32 175MB ...
    va             (time, plev, cell) float32 5GB ...
    vas            (time, cell) float32 175MB ...
    wa             (time, plev, cell) float32 5GB ...
    wap            (time, plev, cell) float32 5GB ...
    zg             (time, plev, cell) float32 5GB ...

In [12]:
def worldmap(var, **kwargs):
    projection = ccrs.Robinson(central_longitude=-135.5808361)
    fig, ax = plt.subplots(
        figsize=(8, 4), subplot_kw={"projection": projection}, constrained_layout=True
    )
    ax.set_global()

    hpshow = egh.healpix_show(var, ax=ax, **kwargs)
    cbar = plt.colorbar(hpshow, ax=ax, orientation='vertical', 
                    pad=0.05, shrink=0.8)
    # fails when cartopy tries to download 
    #ax.add_feature(cf.COASTLINE, linewidth=0.8)
    #ax.add_feature(cf.BORDERS, linewidth=0.4)

In [ ]:
%%time

#worldmap(ds.uas.sel(time=slice('2020-01-01', '2020-12-31')).mean(dim='time'))
worldmap(ds.uas.sel(time=slice('2020-01-01')).mean(dim='time'))

In [ ]:
%%time

# this works with 64 Gb of memory, not sure lower limit
worldmap(dsn.uas.sel(time=slice('2020-01-01', '2020-01-31')).mean(dim='time'))

In [ ]:
%%time

plt.plot(ds.pr.mean("cell"))

In [ ]:
%%time

vt='ua'
tm='2020-01-05'
tmp=ds[vt].sel(time=tm)[0]

tmp

zm = (
    tmp
    .groupby("lat")
    .mean()
).compute()

zm.plot()
